In [ ]:
import os
import warnings

# Train labels generator

In [ ]:
# were to look for files to make the feature extraction
load_path = "/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev"

# output file path
dump_path = "/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/files_directories/labels/labels.ndx"

speakers_set = set()
speakers_dict = {}
for (dir_path, dir_names, file_names) in os.walk(load_path):
    speaker_chunk = [chunk for chunk in dir_path.split("/") if chunk.startswith("id")]
    
    if len(speaker_chunk) > 0: 
        
        speaker_id = speaker_chunk[0]
        if speaker_id not in speakers_set:
            speakers_dict[speaker_id] = {}
            speakers_dict[speaker_id]["files_paths"] = set()
        speakers_set.add(speaker_id)
        
        for file_name in file_names:
            if file_name.split(".")[-1] == "pickle":                
                
                file_path = dir_path + "/" + file_name.replace(".pickle", "")
                speakers_dict[speaker_id]["files_paths"].add(file_path)
        
        if len(speaker_chunk) > 1:
            warnings.warn("Ambiguous directory path!")

speakers_list = list(speakers_set)
speakers_list.sort()
num_speakers = len(speakers_list)
for i, speaker in enumerate(speakers_list):
    speakers_dict[speaker]["speaker_num"] = i
    
speakers_dict = {k: v for k, v in sorted(speakers_dict.items(), key=lambda item: item[1]["speaker_num"])}
    
with open(dump_path, 'w') as f:
    for key, value in speakers_dict.items():
        speaker_num = value["speaker_num"]
        for file_path in value["files_paths"]:
            line_to_write = f"{file_path} {speaker_num} -1"  
            f.write(line_to_write)
            f.write('\n')

# Clients and impostors generator

In [ ]:
import os
import warnings

import itertools

In [ ]:
%%time

# were to look for files to make the feature extraction
load_path = "/home/usuaris/veu/federico.costa/datasets/voxceleb2/dev"

# output file path
impostors_dump_path = "/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/files_directories/labels/impostors.ndx"
clients_dump_path = "/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/files_directories/labels/clients.ndx"

speakers_set = set()
speakers_dict = {}
for (dir_path, dir_names, file_names) in os.walk(load_path):
    speaker_chunk = [chunk for chunk in dir_path.split("/") if chunk.startswith("id")]
    
    if len(speaker_chunk) > 0: 
        
        speaker_id = speaker_chunk[0]
        if speaker_id not in speakers_set:
            speakers_dict[speaker_id] = {}
            speakers_dict[speaker_id]["files_paths"] = set()
        speakers_set.add(speaker_id)
        
        for file_name in file_names:
            if file_name.split(".")[-1] == "pickle":                
                
                file_path = dir_path + "/" + file_name.replace(".pickle", "")
                speakers_dict[speaker_id]["files_paths"].add(file_path)
        
        if len(speaker_chunk) > 1:
            warnings.warn("Ambiguous directory path!")

speakers_list = list(speakers_set)
speakers_list.sort()
num_speakers = len(speakers_list)
for i, speaker in enumerate(speakers_list):
    speakers_dict[speaker]["speaker_num"] = i
    
speakers_dict = {k: v for k, v in sorted(speakers_dict.items(), key=lambda item: item[1]["speaker_num"])}

clients_lines_to_write = []
impostors_lines_to_write = []

distinct_speakers = list(speakers_dict.keys())

one_speaker_combinations = [(speaker, speaker) for speaker in distinct_speakers]
two_speaker_combinations = list(itertools.combinations(distinct_speakers, 2))  
speaker_combinations = one_speaker_combinations + two_speaker_combinations

for speaker_1, speaker_2 in speaker_combinations:
    
    speaker_1_files = speakers_dict[speaker_1]["files_paths"]
    speaker_2_files = speakers_dict[speaker_2]["files_paths"]
    
    if speaker_1 == speaker_2:
        files_combinations = list(itertools.combinations(speaker_1_files, 2))
        for file_1, file_2 in files_combinations:
            line_to_write = file_1 + " " + file_2
            clients_lines_to_write.append(line_to_write)
    else:
        files_combinations = list(itertools.product(speaker_1_files, speaker_2_files))
        for file_1, file_2 in files_combinations:
            line_to_write = file_1 + " " + file_2
            impostors_lines_to_write.append(line_to_write)
            
print(f"{len(clients_lines_to_write)} lines to write for clients.")
print(f"{len(impostors_lines_to_write)} lines to write for impostors.")
            
with open(clients_dump_path, 'w') as f:
    for line_to_write in clients_lines_to_write: 
        f.write(line_to_write)
        f.write('\n')
    f.close()
    
with open(impostors_dump_path, 'w') as f:
    for line_to_write in impostors_lines_to_write: 
        f.write(line_to_write)
        f.write('\n')
    f.close()

In [ ]:
impostors_dump_path

In [ ]:
print(f"{len(clients_lines_to_write)} lines to write for clients.")
print(f"{len(impostors_lines_to_write)} lines to write for impostors.")

In [ ]:
len(clients_lines_to_write)

In [ ]:
len(impostors_lines_to_write)

In [ ]:
impostors_lines_to_write[:10]

In [ ]:
s = 0
for k_1, v_1 in speakers_dict.items():
    for k_2, v_2 in speakers_dict.items():
        if k_1 != k_2:
            s = s + len(v_1["files_paths"]) * len(v_2["files_paths"])
s/2

In [ ]:
s = 0
for k, v in speakers_dict.items():
    s = s + math.comb(len(v["files_paths"]), 2)
s

In [ ]:
len(speakers_dict.keys())

In [ ]:
num_speakers

In [ ]:
s = 0
for k, v in speakers_dict.items():
    s = s + len(v["files_paths"])
s